# EBSI Wallet Conformance (Natural Person)

> EBSI v2 DID for Natural Person(s) is introduced. 

## Setup conformance headers

In [19]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

headers = {
    "Conformance": str(uuid.uuid4()),
}

## Setup wallet

In [20]:
from src.main import wallet

# Setup wallet
client = await wallet("init_v2")


## ISSUE_011

In [21]:
from src.main import conformance
from src.util import parse_query_string_parameters_from_url

result = await conformance("issuerInitiate", headers=headers)

console.log("issuerInitiate", result)

session_token = result.replace("openid://", "")
credential_type = parse_query_string_parameters_from_url(
    session_token).get("credential_type")[0]
issuer_url = parse_query_string_parameters_from_url(
    session_token).get("issuer")[0]

console.log("credential_type", credential_type)

options = {
    "client": client,
    "credential_type" : credential_type
}

result = await conformance("issuerAuthorize", options=options)
console.log("issuerAuthorize", result)


[01:44:07] issuerInitiate openid://initiate_issuance?issuer=https%3A%2F%2Fapi ]8;id=91628;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py\1433138924.py]8;;\:]8;id=807660;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py#6\6]8;;\
           .conformance.intebsi.xyz%2Fconformance%2Fv2&credential_type=https%                
           3A%2F%2Fapi.preprod.ebsi.eu%2Ftrusted-schemas-registry%2Fv2%2Fsche                
           mas%2FzCfNxx5dMBdf4yVcsWzj1anWRuXcxrXj1aogyfN1xSu8t&conformance=5f                
           212e03-e654-43ec-83f3-61b1a786228a                                                

           credential_type https://api.preprod.ebsi.eu/trusted-schemas-regis ]8;id=866903;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py\1433138924.py]8;;\:]8;id=908264;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py#14\14]8;;\
           try/v2/schemas/zCfNxx5dMBdf4yVcsWzj1anWRuXcxrXj1aogyfN1xSu8t                      

[01:44:08] issuerAuthorize                                                   ]8;id=784961;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py\1433138924.py]8;;\:]8;id=92294;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1433138924.py#22\22]8;;\
           {'state': '7c23cbe49775', 'code': 'd15b916e1e73649a061b'}                         

## ISSUE_021

In [22]:
options = {
    "did_version": client.did_version,
    "code": result["code"]
}

result = await conformance("issuerToken", options=options, headers=headers)
console.log("issuerToken", result)


[01:44:09] issuerToken                                                        ]8;id=410646;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1256846934.py\1256846934.py]8;;\:]8;id=966197;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/1256846934.py#7\7]8;;\
           {                                                                                 
               'token_type': 'Bearer',                                                       
               'access_token': '3c0e945758ddd68dc009',                                       
               'expires_in': 3600,                                                           
               'c_nonce': '4a56a26087de157150d2',                                            
               'id_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpYXQiOj                
           E2NTgxNzUyNDksInN1YiI6Imh0dHBzOi8vbG9jYWxob3N0OjMwMDAiLCJhdWQiOiJo                
           dHRwczovL2xvY2FsaG9zdDozMDAwIiwiaXNzIjoiZGlkOmVic2k6emljZnRWUk5ra3                
           dmUm91clpkM25FNWYifQ.BuSN2kl_chHPkf2KKNHWKP8N8U68KVGBNsVcHVmZZ7UyO                
           n4q--k3rVFkzdt8DqcotfWCswO2Kfp-CbCtZ_PNAQ'                                        
           }                                                                                 

## ISSUE_031

In [23]:
from src.did_jwt import decode_jwt

options = {
    "c_nonce": result["c_nonce"],
    "client": client,
    "issuer_url": issuer_url,
    "credential_type": credential_type,
}

access_token = result["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}",
}

result = await conformance("issuerCredential", options=options, headers=headers)
console.log("issuerCredential", result)

credential = decode_jwt(result["credential"])

console.log("Received credential", credential)

[01:44:10] issuerCredential                                                  ]8;id=210907;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/2102501115.py\2102501115.py]8;;\:]8;id=740902;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/2102501115.py#17\17]8;;\
           {                                                                                 
               'format': 'jwt_vc',                                                           
               'credential': 'eyJhbGciOiJFUzI1NksiLCJ0eXAiOiJKV1QiLCJraWQiOi                 
           JkaWQ6ZWJzaTp6aWNmdFZSTmtrd2ZSb3VyWmQzbkU1ZiNrZXlzLTIifQ.eyJqdGki                 
           OiJ1cm46ZGlkOjQ4MmU1YWExLTcyZjgtNGI0Zi1iY2Y2LTQ2ZjdmZTM5ZjgxMyIsI                 
           nN1YiI6ImRpZDplYnNpOnpzd01WeXVydDFhcnpqS0pEejk2WHVHcG5Xb1V2WnphcF                 
           d5VndYVmc5ODhudSIsImlzcyI6ImRpZDplYnNpOnppY2Z0VlJOa2t3ZlJvdXJaZDN                 
           uRTVmIiwibmJmIjoxNjU4MTc1MjUwLCJpYXQiOjE2NTgxNzUyNTAsInZjIjp7IkBj                 
           b250ZXh0IjpbImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxI                 
           l0sImlkIjoidXJuOmRpZDo0ODJlNWFhMS03MmY4LTRiNGYtYmNmNi00NmY3ZmUzOW                 
           Y4MTMiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF                 
           0dGVzdGF0aW9uIiwiVmVyaWZpYWJsZUlkIl0sImlzc3VlciI6ImRpZDplYnNpOnpp                 
           Y2Z0VlJOa2t3ZlJvdXJaZDNuRTVmIiwiaXNzdWFuY2VEYXRlIjoiMjAyMi0wNy0xO                 
           FQyMDoxNDoxMFoiLCJ2YWxpZEZyb20iOiIyMDIyLTA3LTE4VDIwOjE0OjEwWiIsIm                 
           lzc3VlZCI6IjIwMjItMDctMThUMjA6MTQ6MTBaIiwiY3JlZGVudGlhbFN1YmplY3Q                 
           iOnsiaWQiOiJkaWQ6ZWJzaTp6c3dNVnl1cnQxYXJ6aktKRHo5Nlh1R3BuV29Vdlp6                 
           YXBXeVZ3WFZnOTg4bnUiLCJwZXJzb25hbElkZW50aWZpZXIiOiJJVC9ERS8xMjM0I                 
           iwiZmFtaWx5TmFtZSI6IkNhc3RhZmlvcmkiLCJmaXJzdE5hbWUiOiJCaWFuY2EiLC                 
           JkYXRlT2ZCaXJ0aCI6IjE5MzAtMTAtMDEifSwiY3JlZGVudGlhbFNjaGVtYSI6eyJ                 
           pZCI6Imh0dHBzOi8vYXBpLnByZXByb2QuZWJzaS5ldS90cnVzdGVkLXNjaGVtYXMt                 
           cmVnaXN0cnkvdjIvc2NoZW1hcy96Q2ZOeHg1ZE1CZGY0eVZjc1d6ajFhbldSdVhje                 
           HJYajFhb2d5Zk4xeFN1OHQiLCJ0eXBlIjoiRnVsbEpzb25TY2hlbWFWYWxpZGF0b3                 
           IyMDIxIn19fQ.se6jaxViBgCpIVDGOwhMnQPAsj2WgUcJ3NjgIZbqJ7h_iiudrZFN                 
           4348Yq50q9BXoL19pPgaBEI8FSjAif5T0g',                                              
               'c_nonce': 'c627b1f2527edf9cc405',                                            
               'c_nonce_expires_in': 900                                                     
           }                                                                                 

           Received credential                                               ]8;id=165755;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/2102501115.py\2102501115.py]8;;\:]8;id=712800;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_80272/2102501115.py#21\21]8;;\
           {                                                                                 
               'header': {                                                                   
                   'alg': 'ES256K',                                                          
                   'typ': 'JWT',                                                             
                   'kid': 'did:ebsi:zicftVRNkkwfRourZd3nE5f#keys-2'                          
               },                                                                            
               'payload': {                                                                  
                   'jti': 'urn:did:482e5aa1-72f8-4b4f-bcf6-46f7fe39f813',                    
                   'sub':                                                                    
           'did:ebsi:zswMVyurt1arzjKJDz96XuGpnWoUvZzapWyVwXVg988nu',                         
                   'iss': 'did:ebsi:zicftVRNkkwfRourZd3nE5f',                                
                   'nbf': 1658175250,                                                        
                   'iat': 1658175250,                                                        
                   'vc': {                                                                   
                       '@context': [                                                         
                           'https://www.w3.org/2018/credentials/v1'                          
                       ],                                                                    
                       'id': 'urn:did:482e5aa1-72f8-4b4f-bcf6-46f7fe39f813',                 
                       'type': [                                                             
                           'VerifiableCredential',                                           
                           'VerifiableAttestation',                                          
                           'VerifiableId'                                                    
                       ],                                                                    
                       'issuer': 'did:ebsi:zicftVRNkkwfRourZd3nE5f',                         
                       'issuanceDate': '2022-07-18T20:14:10Z',                               
                       'validFrom': '2022-07-18T20:14:10Z',                                  
                       'issued': '2022-07-18T20:14:10Z',                                     
                       'credentialSubject': {                                                
                           'id':                                                             
           'did:ebsi:zswMVyurt1arzjKJDz96XuGpnWoUvZzapWyVwXVg988nu',                         
                           'personalIdentifier': 'IT/DE/1234',                               
                           'familyName': 'Castafiori',                                       
                           'firstName': 'Bianca',                                            
                           'dateOfBirth': '1930-10-01'                                       
                       },                                                                    
                       'credentialSchema': {                                                 
                           'id': 'https://api.preprod.ebsi.eu/trusted-schema                 
           s-registry/v2/schemas/zCfNxx5dMBdf4yVcsWzj1anWRuXcxrXj1aogyfN1xSu                 
           8t',                                                                              
                     